In [10]:
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Conv2D, LeakyReLU,MaxPooling2D,Flatten,Dense,Dropout
from keras.models import Model
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split

## Extract the features

### LeakyReLU 

In [13]:
## VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Replace ReLU with LeakyReLU
x = base_model.input
for layer in base_model.layers:
    if isinstance(layer, Conv2D):
        # Creates a new layer with the same configuration as the current layer, but without the activation function
        new_layer = Conv2D(
            filters=layer.filters,
            kernel_size=layer.kernel_size,
            strides=layer.strides,
            padding=layer.padding,
            activation=None,  
            name=layer.name
        )
        
        new_layer.build(layer.input_shape)
        new_layer.set_weights(layer.get_weights())
        
        x = new_layer(x)
        
        # add LeakyReLU
        x = LeakyReLU(alpha=0.01)(x)
        
    elif isinstance(layer, MaxPooling2D):
        x = layer(x)


# Add the full connection layer
x = Flatten()(x)  
x = Dense(4096)(x)
x = LeakyReLU(alpha=0.01)(x)
x = Dropout(0.5)(x)

# feature vector 4096
LeakyReLU_model = Model(inputs=base_model.input, outputs=x)



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 224, 224, 64)      0         
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 224, 224, 64)      0         
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     7385

In [14]:
#Load images, preprocess, and extract features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = model.predict(img_preprocessed)
    return features.flatten()
    #Extract the features and save the results to a CSV file
def process_images_and_save_features(source_dir, output_csv_file, model):
    data = {'filename': [], 'features': []}
    
    for subdir, dirs, files in os.walk(source_dir):
        category_name = os.path.basename(subdir)
        if files:
            print(f"Processing category: {category_name}")
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, file)
                features = extract_features(img_path, model)
                data['filename'].append(img_path)
                data['features'].append(features)
    

    df = pd.DataFrame(data)
    df['features'] = df['features'].apply(lambda x: ','.join(x.astype(str)))  # Converts an array of features to a comma-separated string
    df.to_csv(output_csv_file, index=False)
    print("Features are successfully saved to CSV.")


source_directory = './after-images'  
output_csv_file = 'LeakyReLU_features.csv'  
process_images_and_save_features(source_directory, output_csv_file, LeakyReLU_model)


Processing category: 001.Black_footed_Albatross
1/1 [==============================] - 0s 150ms/step
Processing category: 002.Laysan_Albatross
1/1 [==============================] - 0s 96ms/step
Processing category: 003.Sooty_Albatross
1/1 [==============================] - 0s 100ms/step
Processing category: 004.Groove_billed_Ani
1/1 [==============================] - 0s 90ms/step
Processing category: 005.Crested_Auklet
1/1 [==============================] - 0s 91ms/step
Processing category: 006.Least_Auklet
1/1 [==============================] - 0s 94ms/step
Processing category: 007.Parakeet_Auklet
1/1 [==============================] - 0s 132ms/step
Processing category: 008.Rhinoceros_Auklet
1/1 [==============================] - 0s 95ms/step
Processing category: 009.Brewer_Blackbird
1/1 [==============================] - 0s 96ms/step
Processing category: 010.Red_winged_Blackbird
1/1 [==============================] - 0s 92ms/step
Processing category: 011.Rusty_Blackbird
1/1 [=======

In [15]:
df = pd.read_csv('LeakyReLU_features.csv')

print(df.head(10))

# Analyze statistical data for features, such as maximum, minimum, and average values
feature_stats = df['features'].apply(lambda x: pd.Series([float(i) for i in x.split(',')]))
print(feature_stats.describe())

                                            filename  \
0  ./after-images\001.Black_footed_Albatross\Blac...   
1  ./after-images\001.Black_footed_Albatross\Blac...   
2  ./after-images\001.Black_footed_Albatross\Blac...   
3  ./after-images\001.Black_footed_Albatross\Blac...   
4  ./after-images\001.Black_footed_Albatross\Blac...   
5  ./after-images\001.Black_footed_Albatross\Blac...   
6  ./after-images\001.Black_footed_Albatross\Blac...   
7  ./after-images\001.Black_footed_Albatross\Blac...   
8  ./after-images\001.Black_footed_Albatross\Blac...   
9  ./after-images\001.Black_footed_Albatross\Blac...   

                                            features  
0  -0.32282764,-0.33388615,-0.3026956,-0.16015178...  
1  -0.5151883,-0.40601066,-0.2200102,-0.104522504...  
2  -0.2909797,-0.3831334,-0.105868734,-0.16712645...  
3  -0.2644592,-0.3867885,-0.20360933,-0.03952134,...  
4  -0.5415482,-0.27620023,-0.35882914,-0.24081586...  
5  -0.23514183,-0.41969904,-0.05220923,-0.3252513... 